Needed Libraries

- Note about installing necessary packages
- Look up proper way to note libraries needed

In [2]:
%load_ext dotenv

In [3]:
# Data Analysis Packages
import numpy as np
import pandas as pd
import matplotlib as mpl
import sklearn

# -- Utility Packages
import bs4

# ---- Folium for Map Generation
import folium

# ---- Packages used for API calls
import requests
import json

# ---- os for drawing out API keys from enviroment variables
import os

from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [10]:
# Retrieve Secret and ID from Enviroment Variables
%dotenv

# Yelp Key and ID
YELP_CLIENT_ID = os.getenv('YELP_CLIENT_ID')
YELP_API_KEY = os.getenv('YELP_API_KEY')

# Code to Retreive Foursquare API
FS_CLIENT_ID = os.getenv('FOURSQUARE_CLIENT_ID')
FS_CLIENT_SECRET = os.getenv('FOURSQUARE_CLIENT_SECRET')
FS_VERSION = '20180323'


### Pull list of Cities from Orange County Website

In [5]:
r = requests.get('https://media.ocgov.com/about/infooc/links/oc/occities.asp')
soup = bs4.BeautifulSoup(r.text)

oc_city_list = []
for cityTag in soup.select('div.edtdiv ul > li div a'):
    city_string = str(cityTag.string)
    # Strip out 'City of'
    city_string = city_string[8:]
    oc_city_list.append(city_string)
oc_city_list

['Aliso Viejo',
 'Anaheim',
 'Brea',
 'Buena Park',
 'Costa Mesa',
 'Cypress',
 'Dana Point',
 'Fountain Valley',
 'Fullerton',
 'Garden Grove',
 'Huntington Beach',
 'Irvine',
 'La Habra',
 'La Palma',
 'Laguna Beach',
 'Laguna Hills',
 'Laguna Niguel',
 'Laguna Woods',
 'Lake Forest',
 'Los Alamitos',
 'Mission Viejo',
 'Newport Beach',
 'Orange',
 'Placentia',
 'Rancho Santa Margarita',
 'San Clemente',
 'San Juan Capistrano',
 'Santa Ana',
 'Seal Beach',
 'Stanton',
 'Tustin',
 'Villa Park',
 'Westminster',
 'Yorba Linda']

### Test API Call and Review Results

#### Foursquare's Places API

In [14]:
url = "https://api.foursquare.com/v2/venues/search"

# Category ID for the Food category on Foursquare
catID = "4d4b7105d754a06374d81259"

params = dict(
    client_id = FS_CLIENT_ID,
    client_secret = FS_CLIENT_SECRET,
    v = FS_VERSION,
    near = 'Aliso Viejo, CA',
    limit = 50,
    categoryId = catID,
    radius = 3600, # Radius is roughly estimated to encapsulate all of Aliso Viejo
)
fs_resp = requests.get(url=url, params=params)
fs_data = json.loads(fs_resp.text)
venues = fs_data['response']['venues']
venues

[{'id': '4ad88efdf964a5204d1221e3',
  'name': 'Dairy Queen',
  'contact': {},
  'location': {'address': '27782 Aliso Creek Rd Ste A100',
   'lat': 33.55928690982983,
   'lng': -117.72477083288777,
   'labeledLatLngs': [{'label': 'display',
     'lat': 33.55928690982983,
     'lng': -117.72477083288777},
    {'label': 'entrance', 'lat': 33.559633, 'lng': -117.72464}],
   'postalCode': '92656',
   'cc': 'US',
   'city': 'Aliso Viejo',
   'state': 'CA',
   'country': 'United States',
   'formattedAddress': ['27782 Aliso Creek Rd Ste A100',
    'Aliso Viejo, CA 92656',
    'United States']},
  'categories': [{'id': '4bf58dd8d48988d1c9941735',
    'name': 'Ice Cream Shop',
    'pluralName': 'Ice Cream Shops',
    'shortName': 'Ice Cream',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/icecream_',
     'suffix': '.png'},
    'primary': True}],
  'verified': True,
  'stats': {'tipCount': 0,
   'usersCount': 0,
   'checkinsCount': 0,
   'visitsCount': 0},
  'delivery': {'i

In [24]:
aliso_viejo_venues = []

for venue in venues:
    if(venue['location']['city'] == 'Aliso Viejo'):
        aliso_viejo_venues.append(venue)
        
aliso_viejo_venues[0]

{'id': '4ad88efdf964a5204d1221e3',
 'name': 'Dairy Queen',
 'contact': {},
 'location': {'address': '27782 Aliso Creek Rd Ste A100',
  'lat': 33.55928690982983,
  'lng': -117.72477083288777,
  'labeledLatLngs': [{'label': 'display',
    'lat': 33.55928690982983,
    'lng': -117.72477083288777},
   {'label': 'entrance', 'lat': 33.559633, 'lng': -117.72464}],
  'postalCode': '92656',
  'cc': 'US',
  'city': 'Aliso Viejo',
  'state': 'CA',
  'country': 'United States',
  'formattedAddress': ['27782 Aliso Creek Rd Ste A100',
   'Aliso Viejo, CA 92656',
   'United States']},
 'categories': [{'id': '4bf58dd8d48988d1c9941735',
   'name': 'Ice Cream Shop',
   'pluralName': 'Ice Cream Shops',
   'shortName': 'Ice Cream',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/icecream_',
    'suffix': '.png'},
   'primary': True}],
 'verified': True,
 'stats': {'tipCount': 0,
  'usersCount': 0,
  'checkinsCount': 0,
  'visitsCount': 0},
 'delivery': {'id': '2039218',
  'url': 'https:

#### Switching to Yelp's API

Restaurant list will be built from calls to Yelp's Fusion API

The Endpoint will be businesses search (https://www.yelp.com/developers/documentation/v3/business_search)

Test API Endpoint with Aliso Viejo

In [21]:
url = "https://api.yelp.com/v3/businesses/search"
headers = dict(
    Authorization = 'Bearer ' + YELP_API_KEY
)
params = dict(
    location = 'Aliso Viejo, CA',
    limit = 50,
    categories = 'Food',
    radius = 4000,
    sort_by = 'distance'
)

resp = requests.get(headers=headers, url=url, params=params)
data = json.loads(resp.text)

In [28]:
businesses = data['businesses']
businesses[0]

{'id': 'pMTnXnGvvT5rbeU_Spc56Q',
 'alias': 'raising-canes-chicken-fingers-aliso-viejo-2',
 'name': "Raising Cane's Chicken Fingers",
 'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/7cHxGA9_2KywFVIjSq-bUA/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/raising-canes-chicken-fingers-aliso-viejo-2?adjust_creative=lNa-mEAkezU89UXKeXq2vQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=lNa-mEAkezU89UXKeXq2vQ',
 'review_count': 438,
 'categories': [{'alias': 'hotdogs', 'title': 'Fast Food'}],
 'rating': 3.5,
 'coordinates': {'latitude': 33.5730323791504, 'longitude': -117.723907470703},
 'transactions': ['delivery'],
 'price': '$',
 'location': {'address1': '26801 Aliso Creek Rd',
  'address2': '',
  'address3': '',
  'city': 'Aliso Viejo',
  'zip_code': '92656',
  'country': 'US',
  'state': 'CA',
  'display_address': ['26801 Aliso Creek Rd', 'Aliso Viejo, CA 92656']},
 'phone': '+19494480241',
 'display_phone': '(949) 448-0241',
 'distance': 170.4

Some of the data wont be used in the analysis, so it can be trimmed off, keeping only:
 - id
 - name
 - review_count
 - categories - title as category
 - rating
 - coordinates
 - price
 - location - city as city

In [25]:
aliso_viejo_spots = []

for business in businesses:
    if(business['location']['city'] == 'Aliso Viejo'):
        trimmed_business = dict(
            id = business['id'],
            name = business['name'],
            review_count = business['review_count'],
            category = business['categories']['title'],
            rating = business['rating'],
            coordinates = business['coordinates'],
            price = business['price'],
            city = business['location']['city']
        )
        aliso_viejo_spots.append(trimmed_business)

{'id': 'pMTnXnGvvT5rbeU_Spc56Q',
 'alias': 'raising-canes-chicken-fingers-aliso-viejo-2',
 'name': "Raising Cane's Chicken Fingers",
 'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/7cHxGA9_2KywFVIjSq-bUA/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/raising-canes-chicken-fingers-aliso-viejo-2?adjust_creative=lNa-mEAkezU89UXKeXq2vQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=lNa-mEAkezU89UXKeXq2vQ',
 'review_count': 438,
 'categories': [{'alias': 'hotdogs', 'title': 'Fast Food'}],
 'rating': 3.5,
 'coordinates': {'latitude': 33.5730323791504, 'longitude': -117.723907470703},
 'transactions': ['delivery'],
 'price': '$',
 'location': {'address1': '26801 Aliso Creek Rd',
  'address2': '',
  'address3': '',
  'city': 'Aliso Viejo',
  'zip_code': '92656',
  'country': 'US',
  'state': 'CA',
  'display_address': ['26801 Aliso Creek Rd', 'Aliso Viejo, CA 92656']},
 'phone': '+19494480241',
 'display_phone': '(949) 448-0241',
 'distance': 170.4

['hi']